# HydroSOS Streamflow Status Product Methodology
#### Jose Valles (jose.valles.leon@gmail.com)

In [1]:
# Import libraries
import os
import sys
import requests
import json 

from IPython.display import HTML


# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

Import HydroSOS_scripts module package \
Make sure that in the path command is defined the folder path of the HydroSOS_script package

In [2]:
# Define the path to the python HydroSOS Package
sys.path.append('..')

import HydroSOS_scripts.get_status_product as SOS
import HydroSOS_scripts.aggregate_variable as aggvar


## Test HydroSOS Package using a single csv file
First, we are going to test the package for a single streamflow location. \
the user must provide the input directory of the file (or files) and the filename

In [28]:
input_directory = '../data/'
filename = '1630.csv'

We will use the import_data function which allows to import data from a csv file \
Recall that the csv file should have three columns: stationid, date, flow, 

In [30]:
if 'waterbalance' in input_directory:
    # Operación si 'waterbalance' está en el input_directory
    print("Realizando la operación para 'waterbalance'")
    MONTHLY_DISCHARGE, stationid = aggvar.import_monthly(input_directory, filename)
    # Aquí puedes agregar la operación que deseas
else:
    # Operación si 'waterbalance' no está en el input_directory
    print("Realizando operación para serie diaria de caudales en estaciones hidrometrica")
    flowdata, stationid = aggvar.import_data(input_directory, filename)
    MONTHLY_DISCHARGE = aggvar.calculate_monthly(flowdata)
    # Aquí puedes agregar la operación alternativa

HTML(MONTHLY_DISCHARGE.tail(12).to_html())

Realizando operación para serie diaria de caudales en estaciones hidrometrica


AttributeError: 'str' object has no attribute 'strftime'

The workflow of this python package is: 
* convert daily to monthly discharge
* calculate the percentage of average based on the monthly climatology
* calculate the rank percentile
* using the Weibull formula, we assign the status categories (1-LowFlow, 2-BelowNormal, 3-Normal, 4-AboveNormal, 5-HighFlow)

In [5]:
MONTHLY_STATUS = SOS.monthly_status(MONTHLY_DISCHARGE)
HTML(MONTHLY_STATUS.tail(12).to_html())

,date,month,year,mean_flow,percentile_flow,rank_average,complete%,weibell_rank,percentile_range,flowcat
525,2023-10-01,10,2023,29.884430,-0.680376,10.0,44.0,0.222222,Below normal,2
526,2023-11-01,11,2023,15.254896,-0.741520,9.0,44.0,0.200000,Below normal,2
527,2023-12-01,12,2023,68.302716,0.980778,38.0,44.0,0.844444,Above normal,4
528,2024-01-01,1,2024,31.180262,0.695349,35.0,45.0,0.760870,Above normal,4
529,2024-02-01,2,2024,14.343698,-0.596178,26.0,45.0,0.565217,Normal range,3
530,2024-03-01,3,2024,361.387668,6.142502,44.0,45.0,0.956522,High flow,5
531,2024-04-01,4,2024,233.044506,4.293583,43.0,45.0,0.934783,High flow,5
532,2024-05-01,5,2024,226.776251,2.511804,43.0,45.0,0.934783,High flow,5
533,2024-06-01,6,2024,148.281664,0.471169,33.0,45.0,0.717391,Normal range,3
534,2024-07-01,7,2024,50.619463,-0.533224,15.0,45.0,0.326087,Normal range,3


In [6]:
output_directory = '../output_csv/01_month/'
filename = stationid

SOS.export_csv(MONTHLY_STATUS,output_directory,filename)

CSV FILE GENERATED


### Calculate the Accumulated Hydrological Status for 3-Months (Quaterly)

After testing the HydroSOS Package and sending the results to Dinagua Web Service, we calculate the accumulated status product

In [7]:
DISCHARGE_THREE_MONTHS = aggvar.calculate_accumulated(MONTHLY_DISCHARGE, 3)
HTML(DISCHARGE_THREE_MONTHS.tail(6).to_html())

,date,startMonth,endMonth,month,year,mean_flow
531,2024-04-01,2,4,4,2024,202.925291
532,2024-05-01,3,5,5,2024,273.736142
533,2024-06-01,4,6,6,2024,202.700807
534,2024-07-01,5,7,7,2024,141.892459
535,2024-08-01,6,8,8,2024,109.738964
536,2024-09-01,7,9,9,2024,79.567909


In [8]:
QUATERLY_STATUS = SOS.quarterly_status(DISCHARGE_THREE_MONTHS)
HTML(QUATERLY_STATUS.tail(6).to_html())

,date,startMonth,endMonth,month,year,mean_flow,percentage_flow,rank_average,complete%,weibell_rank,percentile_range,flowcat,period
531,2024-04-01,2,4,4,2024,202.925291,3.974243,44.0,45.0,0.956522,High flow,5,FMA
532,2024-05-01,3,5,5,2024,273.736142,4.131489,44.0,45.0,0.956522,High flow,5,MAM
533,2024-06-01,4,6,6,2024,202.700807,1.898556,42.0,45.0,0.913043,High flow,5,AMJ
534,2024-07-01,5,7,7,2024,141.892459,0.580126,39.0,45.0,0.847826,Above normal,4,MJJ
535,2024-08-01,6,8,8,2024,109.738964,0.055100,24.0,45.0,0.521739,Normal range,3,JJA
536,2024-09-01,7,9,9,2024,79.567909,-0.215140,15.0,45.0,0.326087,Normal range,3,JAS


### Calculate the Accumulated Hydrological Status for 12-Months (Annualy)

After testing the HydroSOS Package and sending the results to Dinagua Web Service, we calculate the accumulated status product

In [9]:
DISCHARGE_TWELVE_MONTHS = aggvar.calculate_accumulated(MONTHLY_DISCHARGE, 12)
HTML(DISCHARGE_TWELVE_MONTHS.tail(6).to_html())

,date,startMonth,endMonth,month,year,mean_flow
531,2024-04-01,5,4,4,2024,70.116038
532,2024-05-01,6,5,5,2024,88.903716
533,2024-06-01,7,6,6,2024,101.218154
534,2024-07-01,8,7,7,2024,105.420963
535,2024-08-01,9,8,8,2024,113.303619
536,2024-09-01,10,9,9,2024,113.929985


In [10]:
ANNUALY_STATUS = SOS.annualy_status(DISCHARGE_TWELVE_MONTHS)
HTML(ANNUALY_STATUS.tail(6).to_html())

,date,startMonth,endMonth,month,year,mean_flow,percentage_flow,rank_average,complete%,weibull_rank,percentile_range,flowcat
531,2024-04-01,5,4,4,2024,70.116038,0.053876,24.0,44.0,0.533333,Normal range,3
532,2024-05-01,6,5,5,2024,88.903716,0.340311,31.0,44.0,0.688889,Normal range,3
533,2024-06-01,7,6,6,2024,101.218154,0.534485,39.0,44.0,0.866667,Above normal,4
534,2024-07-01,8,7,7,2024,105.420963,0.597024,38.0,44.0,0.844444,Above normal,4
535,2024-08-01,9,8,8,2024,113.303619,0.720563,41.0,44.0,0.911111,High flow,5
536,2024-09-01,10,9,9,2024,113.929985,0.727711,41.0,44.0,0.911111,High flow,5


## Test the HydroSOS Package using all the csv file from an input directory

Now we are going to do the same process for all the csv file in the input directory

In [32]:
input_directory = '../data/'
for f in os.listdir(input_directory):
    if f.endswith('.csv'):
        flowdata, stationid = aggvar.import_data(input_directory, filename = f)
        MONTHLY_DISCHARGE = aggvar.calculate_monthly(flowdata)
        MONTHLY_STATUS = SOS.monthly_status(MONTHLY_DISCHARGE)
        SOS.export_csv(MONTHLY_STATUS, output_directory='../output_csv/01_month/',filename = stationid)
        print("==================================================================")

101
There are 44 years of data in this file.
There are 2441 missing data points, which is 14.93% of the total data
CSV FILE GENERATED
1170
There are 44 years of data in this file.
There are 1201 missing data points, which is 7.35% of the total data
CSV FILE GENERATED
1190
There are 43 years of data in this file.
There are 353 missing data points, which is 2.22% of the total data
CSV FILE GENERATED
1230
There are 43 years of data in this file.
There are 1263 missing data points, which is 7.91% of the total data
CSV FILE GENERATED
1330
There are 44 years of data in this file.
There are 1429 missing data points, which is 8.74% of the total data
CSV FILE GENERATED
140
There are 44 years of data in this file.
There are 2833 missing data points, which is 17.49% of the total data
CSV FILE GENERATED
1410
There are 38 years of data in this file.
There are 1874 missing data points, which is 13.44% of the total data
CSV FILE GENERATED
150
There are 44 years of data in this file.
There are 449 mis

Now, we'll create json files using the csv files from all the stations

In [33]:
SOS.csv_to_json('../output_csv/01_month','../output_json/01_month')

JSON FILE GENERATED


## Send the results to Dinagua Web Service for publishing

After creating the json file for the corresponding year-month, we will update the Web Service using the following process

In [36]:
# Ambiente produccion
url_base = 'https://www.ambiente.gub.uy/dinaguaws/'
# Username and password
user_dinagua = {"user":"FEWS-Uruguay","password":"4MEzOeKFgp0pj9lATSIF"}
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua, verify=False);
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

POST Method for creating the filter Id

In [37]:
# Locate the corresponding json file
json_file = '../output_json/01_month/2024-09.json'
# Define Parameters
date_name = json_file.split('/')[-1].split('.')[0]
cat_scale = 1
ts = "Estaciones"

# create the json dictonary
dataFiltro = {"fecha": date_name,"temporalidad": cat_scale,"serietemporal": ts}

# POST Save Filter
post_filtro = requests.post(url=url_base + "estadohidro/datoscat/guardarFiltro", headers = headers, json=dataFiltro, verify=False)
post_filtro.close()
print(post_filtro.text)

Filtro guardado con éxito


GET Method for extracting the FilterID

In [38]:
# Create the json dictonary
params_filtros = {"fecha" : date_name, "temporalidad" : cat_scale, "serietemporal" : ts}
# Send requests
get_filtro_id = requests.get(url = url_base + "estadohidro/filtros",headers=headers, params=params_filtros, verify=False)
get_filtro_id.close()
# print id
id = json.loads(get_filtro_id.content.decode('utf-8'))
id = id[0]["id"]
print(id)

34


POST HydroSOS Status for the corresponding Month/Year

In [39]:
# Creating the Request Body
datosCat = {"filtrosId": {id}}
# importing the json file from the output directory
json_body = open(json_file)
json_body = json.load(json_body)
# POST HydroSOS status 
post_estado = requests.post(url=url_base+"estadohidro/datoscat/guardarDatosCat",headers=headers,params=datosCat,json=json_body,verify=False)
post_estado.close()
print(post_estado.text)

DatosCat guardado con éxito


We are going to see if the hydrological status is correctly added in the service

In [40]:
params_status = {"serietemporal": ts, "fecha": date_name, "temporalidad": cat_scale}
get_status = requests.get(url=url_base+"estadohidro/datoscat",headers=headers,params=params_status,verify=False)
get_status.close()
# Convert it to json and dataframe
response_status = json.loads(get_status.content.decode('utf-8'))